In [1]:
# plot the audio data
# use popup window
%matplotlib qt
import matplotlib.pyplot as plt

from utils import *

chrip freq 5000, 15000.0
chrip freq 7000, 11000.0


In [2]:
rate = 48000
seconds = 10 / 400
# y = np.concatenate((sine_wave(400)(t), sine_wave(800)(t)))
# y = np.concatenate((sine_wave(400)(t), sine_wave(400)(t)))
# y = np.zeros(int(0.2*rate))

freq = 1000
cycle_time = 1 / freq
cycle_samples = int(cycle_time * rate)
n_cycles = 50
t = np.linspace(0, n_cycles * cycle_time, n_cycles * cycle_samples+1)[:-1]
y = sine_wave(freq)(t)

In [3]:
def gen_chirp(f_0, c, duration, rate):
    f_1 = c * duration + f_0
    print(f"chrip freq {f_0}, {f_1}")

    def phi(t):
        return 2 * np.pi * (c / 2 * t * t + f_0 * t)

    times = np.linspace(0, duration, int(rate * duration)+1)[:-1]

    chirp = np.sin(phi(times))
    chirp_rev = -np.flip(chirp)
    chirp = np.concatenate((chirp, chirp_rev))
    # chirp = np.sin(2 * np.pi * f_0 * times)

    chirp = np.concatenate((chirp, np.zeros(int(0.001*rate))))
    return chirp

chirp1 = gen_chirp(5000, 10000000, 0.001, 48000)
# chirp1 = gen_chirp(7000, 400000, 0.01, 48000)

# plt.plot(t, y)
sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
# empty for 0.2s
empty = np.zeros(int(0.2 * rate))
audio = np.concatenate((sine_wav, empty, chirp1, y, np.zeros(int(0.5*rate))))
# audio = np.concatenate((sine_wav, empty, y, np.zeros(int(0.5*rate))))

record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)
record_wave = record_wave[int(1*rate):]

def find_start(wave):
    corr1 = np.correlate(wave, chirp1, mode='full') / len(chirp1)
    xmax = np.argmax(corr1)
    return xmax

s1 = -record_wave
# s1 = audio

s1 = s1 / np.max(np.abs(s1))

xmax = find_start(s1)

result_signal = s1[xmax:xmax+len(y)]
result_signal = result_signal / np.max(np.abs(result_signal))

plt.plot(s1)

plt.axhline(y=0, color='k')

corr1 = np.correlate(s1, chirp1, mode='full') / len(chirp1)
plt.plot(corr1)

plt.axvline(x=xmax, color='r')
plt.plot(np.linspace(0, len(chirp1)+len(y), len(chirp1)+len(y), endpoint=False)-len(chirp1) + xmax, np.concatenate((chirp1, y)) * 0.5 + 2)
plt.show()

fft_result = np.fft.fft(result_signal)[:10]
# plot_fft(result_signal, rate)
# plt.plot(np.abs(fft_result))
# plt.plot(np.real(fft_result))
# plt.plot(np.imag(fft_result))
# plt.legend(['abs', 'real', 'imag'])

# coss = []
# sins = []

# for i in range(len(y) // cycle_samples):
#     signal = result_signal[i*cycle_samples:(i+1)*cycle_samples]
#     fft_result = np.fft.fft(signal)[:10]
#     cos_part = np.abs(np.real(fft_result[1]))
#     sin_part = np.abs(np.imag(fft_result[1]))
#     coss = np.append(coss, cos_part)
#     sins = np.append(sins, sin_part)
#     if cos_part > sin_part:
#         print(f"{cos_part > sin_part}")

chrip freq 5000, 15000.0


The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x5a2829efefa0


In [4]:
def gen_chirp(f_0, c, duration, rate):
    f_1 = c * duration + f_0
    print(f"chrip freq {f_0}, {f_1}")

    def phi(t):
        return 2 * np.pi * (c / 2 * t * t + f_0 * t)

    times = np.linspace(0, duration, int(rate * duration)+1)[:-1]

    chirp = np.sin(phi(times))
    chirp_rev = -np.flip(chirp)
    chirp = np.concatenate((chirp, chirp_rev))

    chirp = np.concatenate((chirp, np.zeros(48)))
    return chirp

chirp1 = gen_chirp(5000, 5000000, 0.001, 48000)
# chirp1 = gen_chirp(7000, 400000, 0.01, 48000)

# plt.plot(t, y)
sine_wav = sine_wave_samples(440, 1 * 440, 0, 1, rate)
# empty for 0.2s
empty = np.zeros(int(0.2 * rate))
audio = np.concatenate((sine_wav, empty, chirp1, y, chirp1, y, chirp1, y, chirp1, y, np.zeros(int(0.5*rate))))
# audio = np.concatenate((sine_wav, empty, y, np.zeros(int(0.5*rate))))

record_wave, play_wave, rate = play_and_record(audio, rate, discard=False)
record_wave = record_wave[int(1*rate):]

record_wave = -record_wave

chrip freq 5000, 10000.0


In [7]:
# plt.plot(audio)
save_wave_file("chirp_test_audio.wav", rate, audio)
print('phy payload len', len(y))
print('chirp len')

phy payload len 2400
chirp len 144


In [24]:
# plt.plot(record_wave)

def corr_sliding_window(wave):
    chirp = chirp1
    chirp_len = len(chirp)

    result = np.zeros(len(wave))
    for i in range(0, len(wave)):
        start = max(i - chirp_len + 1, 0)
        end = i
        window = wave[start:end+1]
        if len(window) != chirp_len:
            result[i] = 0
            continue
        window = np.copy(window) / np.max(np.abs(window))
        corr = np.correlate(window, chirp, mode='valid') / len(chirp)
        assert len(corr) == 1
        result[i] = corr[0]
    return result

def chirp_location_sliding_window(corr):
    window_size = 129
    result = np.zeros(len(corr))
    for i in range(len(corr)):
        start = max(i - (window_size-1)//2, 0)
        end = i + (window_size-1)//2
        window = corr[start:end+1]
        if len(window) != window_size:
            window = np.concatenate((np.zeros(window_size - len(window)), window))
        position = np.argmax(window)
        if position == window_size // 2 and corr[i] > 0.1:
            result[i] = 1
    return result

corr = corr_sliding_window(record_wave)
chirp_location = chirp_location_sliding_window(corr)

plt.plot(record_wave)
plt.plot(corr)
# plt.plot(chirp_location, 'o')
for loc in np.where(chirp_location == 1)[0]:
    plt.axvline(x=loc, color='r')
    plt.axvline(x=loc+len(y), color='g')

The cached device pixel ratio value was stale on window update.  Please file a QTBUG which explains how to reproduce.
qt.qpa.wayland.textinput: virtual void QtWaylandClient::QWaylandTextInputv3::zwp_text_input_v3_leave(wl_surface*) Got leave event for surface 0x0 focused surface 0x562349561f90


In [49]:
# corr[0:100]
# plt.plot(corr)
i = 300
start = i - 64
end = i + 64
# len(corr[start:end+1])
print(corr[start:end+1][64])
print(corr[i])

-0.006393047857937883
-0.006393047857937883
